# PYRAWS - Create a geographic map of the dataset

This notebook is to create a geographic map of a target dataset. 

# 1) Imports, paths and variables

Limit CUDA visible devices.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

Autoreload.

In [ ]:
%load_ext autoreload
%autoreload 2

Imports.

In [ ]:
import sys
sys.path.insert(1, os.path.join("..", ".."))

from geopy.geocoders import Nominatim
import geopandas
from geopandas import GeoSeries
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
from pyraws.raw.raw_event import Raw_event
from pyraws.utils.database_utils import get_events_list
from pyraws.utils.database_utils import DATABASE_FILE_DICTIONARY, get_cfg_file_dict
from shapely.geometry import Point
from termcolor import colored
import torch
from tqdm import tqdm

This import is to remove odd errors on `libiomp5md.dll`. If you do not have them, you can skip it

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

Set torch device. Use "CUDA" as default if available.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Set size of figure plots.

In [ ]:
plt.rcParams["figure.figsize"] = [10, 10]

Select target database and target bands.

In [ ]:
dataset="THRAWS" # target dataset
requested_bands = ["B8A"] #Specify one band to make the loading quicker. The band used is irrelevant.
raw_dir=os.path.join(get_cfg_file_dict()["data"], dataset, "raw") # Raw dir in the target database

Select color maps for the events. You should have a color for each class, excluding "not-events".

In [ ]:
colors=["red", "orange", "yellow", "green", "blue"] #color list

# 2) - Loading database and granules

Loading database of the target dataset.

In [ ]:
database=pd.read_csv(os.path.join("..", "..","pyraws", "database", DATABASE_FILE_DICTIONARY[dataset]))

database_events=get_events_list()
events_legend=np.unique([str(x) for x in database["class"]])
events_legend=list(events_legend)

color_event_class_dict=dict(zip(events_legend, colors[:len(colors)])) # color - event_class dictionary

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# events_list = get_events_list("THRAWS")
database_files = sorted(glob(os.path.join(raw_dir, "*")))

# events lift
events_list = [file.split(os.sep)[-1] for file in database_files]

granule_coordinates_list=[]

k=0

for event, file in tqdm(zip(events_list, database_files), "Accessing event..."):
    print("Processing event: ", colored(event, "blue") + ".")
    try:
        raw_event = Raw_event(device=device)
        raw_event.from_path(file, requested_bands, verbose=False)
    except:  # noqa E722
        print("Skipping event: ", colored(event, "red") + ".")
        continue

    if raw_event.is_void():
        print("Skipping event: ", colored(event, "red") + ".")
        continue

    granules_list = list(range(len(raw_event.get_granules_info().keys())))

    for granule in granules_list:
        raw_granule_n = raw_event.get_granule(granule)
        granule_coordinates=np.array(raw_granule_n.get_granule_info()[-2])
        granule_baricenter=granule_coordinates[0] + (granule_coordinates[2] - granule_coordinates[0])/2
        
        if event in database_events:
            useful_granules=[int(x) for x in str(database[database["ID_event"] == event]["Raw_useful_granules"]).split("[")[1].split("]")[0].split(",")]
            if granule in useful_granules:
                event_class=[str(x) for x in database[database["ID_event"] == event]["class"]][0]
                if event_class != "not_event":
                    granule_coordinates_list.append([granule_baricenter, event_class])

print("processing " + colored("finished", "green") + ".")

# 3) - Creating geographical distribution map

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))
world = world.to_crs("EPSG:4326")
geolocator = Nominatim(user_agent="google")
point_color_list=[]
for granule_object in granule_coordinates_list:
    granule_baricenter=granule_object[0]
    coordinates = [granule_baricenter[1], granule_baricenter[0]]
    point = GeoSeries([Point(coordinates)])
    point = point.set_crs("EPSG:4326")
    point_color_list.append([point, color_event_class_dict[granule_object[1]]])
ax = world.plot()

for point, color in point_color_list:
    point.plot(facecolor=color, edgecolor=color, ax=ax)
ax.legend(list(color_event_class_dict.keys())[:-1])
leg = ax.get_legend()

# Fix colors in the legend
for n in range(len(events_legend[:-1])):
    leg.legendHandles[n].set_color(list(color_event_class_dict.values())[n])
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

# 4) - Save geographical distribution map to a figure

In [ ]:
fig = ax.get_figure()
fig.savefig(dataset + "_geo_map.png", dpi=500, bbox_inches='tight')